In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)  # Adjust output layer

    def forward(self, x):
        return self.model(x)

In [3]:
# Transformations: Resize, Normalize
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to fixed size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [4]:
# Caricamento del modello
model_path = './handwriting_model.pth'
#model = torch.load(model_path, map_location=torch.device('cpu'))
#model.eval()

# Load Dataset
data_dir = "./data"
dataset = ImageFolder(root=data_dir, transform=transform)

num_classes = len(dataset.classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = SimpleCNN(num_classes=num_classes).to(device)
model.load_state_dict(torch.load("handwriting_model.pth", map_location=torch.device('cpu')))
model.eval()  # Modalità di valutazione

c:\Users\Giuseppe\anaconda3\envs\bio_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Giuseppe\anaconda3\envs\bio_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SimpleCNN(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data  # I dati sono già nel formato corretto (tensori di immagini)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image, label = self.data[idx]  # Ogni elemento è una tupla (immagine, etichetta)
        return image, label


In [6]:
# Carica il dataset da un file .pth
data = torch.load("val_dataset.pth") 

# Crea il dataset personalizzato
test_dataset = CustomDataset(data)

# Crea il DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize lists to store predictions and labels
all_labels = []
all_preds = []
all_probs = []

# Evaluate the model on the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to("cuda" if torch.cuda.is_available() else "cpu")
        labels = labels.to("cuda" if torch.cuda.is_available() else "cpu")

        # Get model predictions
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # For multi-class models
        _, preds = torch.max(probabilities, 1)

        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probabilities[:, 1].cpu().numpy())  # For binary classification

# Compute metrics
precision = precision_score(all_labels, all_preds, average="weighted")
recall = recall_score(all_labels, all_preds, average="weighted")
f1 = f1_score(all_labels, all_preds, average="weighted")

# ROC and AUC
#fpr, tpr, _ = roc_curve(all_labels, all_probs)
#roc_auc = auc(fpr, tpr)

# Genuine Acceptance Rate (GAR) and Genuine Rejection Rate (GRR)
#genuine_acceptance_rate = tpr[-1]  # GAR: last value of TPR in ROC
#genuine_rejection_rate = 1 - fpr[-1]  # GRR: complement of last FPR in ROC

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)

# Return all metrics
print("Precision: "+ str(precision),
    "\nRecall: " + str(recall),
    "\nF1 Score: " + str(f1),
    "\nConfusion Matrix: " + str(cm))

Precision: 0.9274891774891774 
Recall: 0.935064935064935 
F1 Score: 0.9283549783549784 
Confusion Matrix: [[16  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  4  0]
 [ 0  0  0 ...  0  0  3]]


c:\Users\Giuseppe\anaconda3\envs\bio_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Giuseppe\anaconda3\envs\bio_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [58]:
# Return all metrics
print("Precision: "+ str(precision),
    "\nRecall: " + str(recall),
    "\nF1 Score: " + str(f1),
    "\nConfusion Matrix: " + str(cm))

Precision: 0.9274891774891774 
Recall: 0.935064935064935 
F1 Score: 0.9283549783549784 
Confusion Matrix: [[16  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  4  0]
 [ 0  0  0 ...  0  0  3]]


In [36]:
for batch in test_loader:
    print(f"Tipo: {type(batch)}, Lunghezza: {len(batch)}")
    for i, item in enumerate(batch):
        print(f"Elemento {i}: Tipo {type(item)}, Dimensioni: {item.size() if hasattr(item, 'size') else 'Non Tensor'}")
    break

Tipo: <class 'list'>, Lunghezza: 15
Elemento 0: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 1: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 2: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 3: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 4: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 5: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 6: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 7: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 8: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 9: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 10: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 11: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 12: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 13: Tipo <class 'str'>, Dimensioni: Non Tensor
Elemento 14: Tipo <class 'str'>, Dimensioni: Non Tensor


In [ ]:
# Funzione per calcolare le metriche
def evaluate_model(model, test_loader):
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to("cuda" if torch.cuda.is_available() else "cpu")
            labels = labels.to("cuda" if torch.cuda.is_available() else "cpu")

            # Ottieni le predizioni e le probabilità
            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)
            _, preds = torch.max(probabilities, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probabilities[:, 1].cpu().numpy())  # Supponendo classificazione binaria

    # Precision, Recall, F1 Score
    precision = precision_score(all_labels, all_preds, average="binary")
    recall = recall_score(all_labels, all_preds, average="binary")
    f1 = f1_score(all_labels, all_preds, average="binary")

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)

    # ROC e AUC
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)

    # GAR e GRR
    genuine_acceptance_rate = tpr[-1]  # Ultimo punto della curva ROC
    genuine_rejection_rate = 1 - fpr[-1]

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Confusion Matrix": cm,
        "ROC AUC": roc_auc,
        "GAR": genuine_acceptance_rate,
        "GRR": genuine_rejection_rate,
    }

# Calcolo delle metriche
metrics = evaluate_model(model, test_loader)


In [ ]:
# Stampa dei risultati
print("Metriche di Valutazione:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")